In [3]:
%use krangl
val daten = DataFrame.readCSV("../daten/virusErde14.csv") //Liest die csv-Datei und wandelt sie in einen DataFrame um

# Aufgabe - Komplexe Lambda-Ausdrücke
In der Spalte *Case_Fatality_Ratio* ist der Anteil der verstorbenen Personen an der Gesamtzahl der bestätigten Fälle zu finden. Ihre Aufgabe ist es, eine Erweiterungsmethode `gibGeringsteSterblichkeit()` der Klasse `DataFrame` zu implementieren, die die 10 Länder mit der geringsten Sterblichkeit zurückgibt. Diese soll aus Paaren bestehen. Jedes Paar soll zuerst das Land und dann die Sterblichkeit beinhalten. Um fehlerhaften Daten entgegenzuwirken, sollen nur Länder, die mehr als 20000 bestätigte Fälle aufweisen, ausgewählt werden. Außerdem soll das Land *Belgium* nicht berücksichtigt werden.

In [198]:
fun DataFrame.gibGeringsteSterblichkeit() = this.rows.filter { it["Case_Fatality_Ratio"] as String != "" && it["Confirmed"] as Int > 50000 &&  it["Country_Region"] as String != "Belgium"} //Filtert alle Zeilen heraus, die keinen Eintrag in der Spalte Incident_Rate aufweisen
    .groupBy { it["Country_Region"] }   //Gruppiert die Einträge nach dem Land. Zurückgegeben wird eine Map mit dem Namen des Landes als Schlüssel und einer Liste, die alle Zeilen des Landes umfasst. 
                                        //Jeder Eintrag in der Liste ist jedoch wieder eine Map, die als Schlüssel den Spaltennamen und als Wert den Eintrag an der jeweiligen Stelle besitzt.
    .mapValues { //Iteriert über die Map und setzt jeden Wert eines Paares auf die Rückgabe des Lambda-Ausdrucks
        val inzidenzen = MutableList<Double>(0) { 0.0 } //Erstellt für jedes Land eine leere Liste, die mit den Inzidenzraten gefüllt werden soll
        it.value.forEach{ //Iteriert über alle Einträge jedes Land
            inzidenzen.add((it["Case_Fatality_Ratio"] as String).toDouble()) //Fügt der Liste die Inzidenzrate der Region hinzu
        }  
        inzidenzen.average() //Berechnet den Durchschnitt der Wert der Liste, wandelt ihn in Prozent um und gibt ihn zurück
    }
    .toList() //Konvertiert die Map in eine Liste aus Paaren, welche aus dem Schlüssel und dem berechneten Wert besteht, da eine Map nicht sortiert werden kann
    .sortedBy { (schluessel, wert) -> wert} //Sortiert die Liste nach den Werten absteigend
    .take(10) //Löscht alle Einträge abgesehen von den ersten Zehn

//Test
val testErgebnis = listOf( 
    Pair("Singapore", 0.049542557056511546), Pair("Qatar", 0.17185729399949518), Pair("United Arab Emirates", 0.319314985326216), 
    Pair("Bahrain", 0.3600318565664677), Pair("Malaysia", 0.36787523838541136), Pair("Cuba", 0.5351024805995085), Pair("Kuwait", 0.5710878000902807), 
    Pair("Sri Lanka", 0.6261314024065595), Pair("Norway", 0.6756890445380977), Pair("Belarus", 0.7000750296488129)
)
val ergebnis = daten.gibGeringsteSterblichkeit()
var fehler = 0
for(i in 0 until testErgebnis.size){
    if(testErgebnis[i] != ergebnis[i]){
        println("Falsch! Eintrag $i sollte ${testErgebnis[i]} sein, ist aber ${ergebnis[i]}")
        fehler++
    }
}
if(fehler == 0)
    println("Super! Alle Tests bestanden!")
else
    println("Bei $fehler Tests kamen Fehler auf.")

Super! Alle Tests bestanden!
